In [1]:
import numpy as np

In [2]:
class Game():
  def __init__(self, player, opponent, turns):
    
    player.clear_moves()
    opponent.clear_moves()
    
    self.player = player
    self.opponent = opponent

    self.score = [0, 0, 0]
    self.turns = turns
    self.turn = 0
    self.dictionary = {0: 'rock', 1: 'scissors', 2: 'paper'}

  def change_score(self, move_1, move_2):
    if move_1 == move_2:
      self.score[1] += 1
    else:
      if move_1 == 0:
        if move_2 == 1:
          self.score[0] += 1
        else:
          self.score[2] += 1
      elif move_1 == 1:
        if move_2 == 0:
          self.score[2] += 1
        else:
          self.score[0] += 1
      elif move_1 == 2:
        if move_2 == 0:
          self.score[0] += 1
        else:
          self.score[2] += 1

  def who_won(self):
    score_string = '-'.join([str(num) for num in self.score])
    if self.turn < self.turns:
      print(f'current score is: {score_string}')
    else:
      print(f'final score is: {score_string}')
      if self.score[0] > self.score[2]:
        print('First player won')
      elif self.score[0] < self.score[2]:
        print('Second player won')
      else:
        print('Draw')

  def game_turn(self):
    move_1 = self.player.move()
    move_2 = self.opponent.move()
    print(f'{self.dictionary[move_1]} - {self.dictionary[move_2]}')
    self.change_score(move_1, move_2)
    self.who_won()

  def play_game(self):
    while self.turn < self.turns:
      self.turn += 1
      self.game_turn()

In [3]:
class Player():
  def __init__(self):
    self.moves = []

  def move(self):
    pass

  def clear_moves(self):
    self.moves = []

In [4]:
class You(Player):
  def __init__(self):
    super().__init__()

  def move(self):
    print("Please make your move:")
    step = int(input("0 - rock, 1 - scissors, 2 - paper"))
    self.moves.append(step)
    return step

In [5]:
class YouPredefined(Player):
  def __init__(self, choice_list):
    super().__init__()
    self.step_n = 0
    self.choice_list = choice_list
    
  def move(self):
    step = self.choice_list[self.step_n]
    self.step_n += 1
    self.moves.append(step)
    return step

In [6]:
class RandomPlayer(Player):
  def __init__(self):
    super().__init__()

  def move(self):
    step = np.random.randint(0, 3)
    self.moves.append(step)
    return step

In [7]:
class AlmostRandomPlayer(Player):
  def __init__(self, *args):
    super().__init__()
    self.num_list = [0 for _ in range(args[0])] + \
    [1 for _ in range(args[1])] + \
    [2 for _ in range(args[2])]
  
  def move(self):
    choice = np.random.choice(self.num_list)
    self.moves.append(choice)
    return choice

In [8]:
class ConstantPlayer(Player):
  def __init__(self, number: int):
    super().__init__()
    self.number = number
    
  def move(self):
    self.moves.append(self.number)
    return self.number

In [9]:
class SlidingPlayer(Player):
  def __init__(self, start_num: int):
    super().__init__()
    self.num = start_num - 1

  def move(self):
    self.num = (self.num + 1) % 3
    self.moves.append(self.num)
    return self.num

In [10]:
class QuadSlidingPlayer(Player):
  def __init__(self, start_num: int):
    super().__init__()
    self.num = start_num
    self.turn = 0
    self.round = 1
    self.length = self.round ** 2

  def move(self):
    if self.length > self.turn:
      self.turn += 1
    else:
      self.turn = 1
      self.round += 1
      self.length = self.round ** 2
      self.num = (self.num + 1) % 3

    self.moves.append(self.num)
    return self.num

In [11]:
class Predictor():
  def __init__(self, player: Player, opponent: Player, model):
    self.player = player
    self.opponent = opponent
    self.model = model

  def process_moves(self):
    pl_moves = self.player.moves.copy()
    opp_moves = self.opponent.moves.copy()

    length = len(pl_moves)

    X = np.ones((length, length)) * -1
    y = np.zeros((length-1, 3))

    for i in range(length):
      X[i][:i+1] = pl_moves[:i+1][::-1]

    for i in range(length-1): 
      if pl_moves[i] == 0:
        y[i][2] = 1
      elif pl_moves[i] == 1:
        y[i][0] = 1
      else:
        y[i][1] = 1
    return X, y

  def fit_predict(self):
    X, y = self.process_moves()
    self.model.fit(X[:-1], y)
    pred = self.model.predict([X[-1]])

    return np.argmax(pred)

In [12]:
class SmartOpponent(Player):
  def __init__(self, opponent: Player, model):
    super().__init__()

    self.opponent = opponent
    self.predictor = Predictor(self, opponent, model)
  
  def move(self):
    if len(self.opponent.moves) < 5:
      step = np.random.randint(0, 3)
    else:
      step = self.predictor.fit_predict()
    
    self.moves.append(step)
    return step

  def change_opponent(self, new_opponent):
    self.opponent = new_opponent

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
model = LinearRegression()

In [ ]:
# Game on local machine

turns = 25

you = You() # You == YouLKA!
smart = SmartOpponent(you, model)

game = Game(you, smart, turns)
game.play_game()

In [16]:
class GameWithoutComments(Game):
  def __init__(self, player, opponent, turns):
    super().__init__(player, opponent, turns)
  
  def game_turn(self):
    move_1 = self.player.move()
    move_2 = self.opponent.move()
    self.change_score(move_1, move_2)

In [ ]:
# Play against Telegram bot (you can create one with support of @BotFather)

In [18]:
!pip install pyTelegramBotAPI
import telebot

     |████████████████████████████████| 104 kB 38.7 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.8.3-py3-none-any.whl size=79359 sha256=c3b474b667c9936f3e13bdb94eb3e2a7ef7c8d3a3c54f5698d871b7c34652cf4
  Stored in directory: /root/.cache/pip/wheels/9c/ac/0f/3d7f8a376c680f4605b35c6d7c2db2180fdeff4a3646d282c2
Successfully built pyTelegramBotAPI


In [19]:
class TeleYou(Player):
  def __init__(self, bot):
    super().__init__()

  def move(self, inp):
    step = int(inp)
    self.moves.append(step)
    return step

In [20]:
class TeleGame(Game):
  def __init__(self, player, opponent, turns):
    super().__init__(player, opponent, turns)
    self.bot = bot

  def game_turn(self, inp: int, chat_id):
    self.message_id = chat_id

    move_1 = self.player.move(inp)
    move_2 = self.opponent.move()
    self.bot.send_message(self.message_id, f'{self.dictionary[move_1]} - {self.dictionary[move_2]}')
    self.change_score(move_1, move_2)
    self.who_won()

  def who_won(self):
    score_string = '-'.join([str(num) for num in self.score])
    if self.turn < self.turns:
      self.bot.send_message(self.message_id, f'current score is: {score_string}')
    else:
      self.bot.send_message(self.message_id, f'final score is: {score_string}')
      if self.score[0] > self.score[2]:
        self.bot.send_message(self.message_id, 'First player won')
      elif self.score[0] < self.score[2]:
        self.bot.send_message(self.message_id, 'Second player won')
      else:
        self.bot.send_message(self.message_id, 'Draw')

In [31]:
bot = telebot.TeleBot('Bot token from BotFather')

you = TeleYou(bot)
opponent = SmartOpponent(you, LinearRegression())
game = TeleGame(you, opponent, 20)

In [32]:
@bot.message_handler(commands=['0', '1', '2', 'rock', 'paper', 'scissors', 'start'])
def start_command(message):
  chat_id = message.chat.id
  if message.text == '/start':
    bot.send_message(chat_id, "Hello. Let's start the game")
    bot.send_message(chat_id, "0 - rock, 1 - paper, 2 - scissors")
    game.score = [0, 0, 0]

  elif message.text == "/0" or message.text == "/rock":
    game.game_turn(0, chat_id)
  elif message.text == "/1" or message.text == "/paper":
    game.game_turn(1, chat_id)
  elif message.text == "/2" or message.text == "/scissors":
    game.game_turn(2, chat_id)

bot.polling()